In [69]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [70]:
colab_path = 'ScadaData.txt'
local_path = 'C:\\Users\\hanna\\Desktop\\data\\ScadaData.txt'

scada_data = pd.read_csv(local_path, delimiter='\t',  parse_dates=True)
scada_data['dtTimeStamp'] = pd.to_datetime(scada_data['dtTimeStamp'])

turbine_names = scada_data['intObjectId'].unique()[0:4]
print(turbine_names)
print("Number of turbines: " + str(len(turbine_names)))

[1748 1749 1750 1751]
Number of turbines: 4


In [71]:
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

now = dt.datetime.now()
wind_direction_data = scada_data[['dtTimeStamp','WindDirectionMean', 'intObjectId']]
wind_direction_data['dtTimeStamp'] = pd.to_datetime(scada_data['dtTimeStamp'])
wind_direction_data.index=wind_direction_data['dtTimeStamp']
print(wind_direction_data.describe())

# plt.figure(figsize=(20,5))
# for turbine in turbine_names:
#     plt.plot(wind_direction_data[wind_direction_data['intObjectId'] == turbine]["WindDirectionMean"], label=turbine)
# plt.legend(loc='best')
# plt.title('Raw Data')

# plt.plot(wind_direction_data[wind_direction_data['intObjectId'] == 1770]["WindDirectionMean"], label=turbine)



       WindDirectionMean    intObjectId
count      235816.000000  237195.000000
mean          185.336724    1761.148148
std           105.440429       8.040211
min             0.000000    1748.000000
25%            83.500000    1754.000000
50%           199.100000    1761.000000
75%           272.900000    1768.000000
max           359.000000    1776.000000


In [72]:
from statsmodels.tsa.arima_model import ARIMA
import datetime as dt

def forecast_wind_direction(data, n_steps, train_test_split, samples_count):
    n_step = n_steps
    X = data[0:samples_count]
    size = int(len(X) * train_test_split)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train.values]
    predictions = list()
    test_indices = [i for i in range(0, len(test.values)) if i%n_step==0]
    for i in test_indices:
        model = ARIMA(history, order=(1,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast(n_step)
        yhat = output[0].flatten()
        predictions.extend(yhat)
        history.extend(test.values[i:i+n_step])
    
    return predictions


samples_to_use = 500
train_test_split = 0.04
n_steps = 1
filtered = wind_direction_data[wind_direction_data['intObjectId'].isin(turbine_names)]
averaged = filtered[['WindDirectionMean', 'intObjectId']].groupby('dtTimeStamp').mean()[['WindDirectionMean']].dropna()
direction_predictions_df = pd.DataFrame(index=averaged[int(samples_to_use * train_test_split):samples_to_use].index)
average_mean_wind_directions_real = averaged[0:samples_to_use]


for turbine in turbine_names:
    print('starting turbine: ' + str(turbine))
    data = wind_direction_data[wind_direction_data['intObjectId'] == turbine]["WindDirectionMean"]  
    predictions = forecast_wind_direction(data, n_steps, train_test_split, samples_to_use)
    direction_predictions_df[turbine] = predictions
    print('finished turbine: ' + str(turbine))
    
print(direction_predictions_df)


starting turbine: 1748
finished turbine: 1748
starting turbine: 1749
finished turbine: 1749
starting turbine: 1750
finished turbine: 1750
starting turbine: 1751
finished turbine: 1751
                           1748        1749        1750        1751
dtTimeStamp                                                        
2019-08-01 03:20:00   79.815823   71.791567   94.828070   82.448571
2019-08-01 03:30:00   82.353939   73.211785  100.630836   85.968380
2019-08-01 03:40:00   81.649696   75.739335  102.562786   88.555278
2019-08-01 03:50:00   79.395588   74.591889   99.037503   86.260820
2019-08-01 04:00:00   79.299663   72.198514   96.183301   83.781151
...                         ...         ...         ...         ...
2019-08-04 10:30:00  184.563532  236.751991  248.660881  258.744404
2019-08-04 10:40:00  174.459101  169.559431  261.370603  261.438919
2019-08-04 10:50:00  160.812311  151.241868  213.070973  232.323796
2019-08-04 11:00:00  168.814517  156.650200  190.028835  180.852684


In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import datetime as dt

wind_speed_data = scada_data[['dtTimeStamp','WindSpeedMean', 'intObjectId']]
wind_speed_data['dtTimeStamp'] = pd.to_datetime(scada_data['dtTimeStamp'])
wind_speed_data.index=wind_speed_data['dtTimeStamp']
print(wind_speed_data.describe())

#wind_speed_data[wind_speed_data['intObjectId'] == 1748 & wind_speed_data['WindSpeedMean'].isnull()]

def forecast_wind_speed(data, column_name):
    X = data[[column_name]]
    size = int(len(X) * 0.1)
    train, test = X[0:size], X[size:len(X)]
    history = train
    predictions = pd.DataFrame(index=test.index, columns=[column_name])
    for i, row in test.iterrows():
        model = ARIMA(history, order=(1,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0].flatten()[0]
        predictions.loc[i][column_name] = yhat
        history.loc[i] = row[column_name]


samples_to_use = 500
train_test_split = 0.04
n_steps = 1
filtered = wind_speed_data[wind_speed_data['intObjectId'].isin(turbine_names)]
averaged = filtered[['WindSpeedMean', 'intObjectId']].groupby('dtTimeStamp').mean()[['WindSpeedMean']].dropna()
speed_predictions_df = pd.DataFrame(index=averaged[int(samples_to_use * train_test_split):samples_to_use].index)
average_mean_wind_speeds_real = averaged[0:samples_to_use]

for turbine in turbine_names:
    
    try:
        print('starting turbine: ' + str(turbine))
        data = wind_speed_data[wind_speed_data['intObjectId'] == turbine] 
        predictions = forecast_wind_speed(data, "WindSpeedMean")
        speed_predictions_df[turbine] = predictions
        print('finished turbine: ' + str(turbine))
    except:
        continue
    
print(speed_predictions_df)


       WindSpeedMean    intObjectId
count  235816.000000  237195.000000
mean        5.231695    1761.148148
std         2.194863       8.040211
min         0.200000    1748.000000
25%         3.600000    1754.000000
50%         5.000000    1761.000000
75%         6.700000    1768.000000
max        19.400000    1776.000000
starting turbine: 1748
starting turbine: 1749
starting turbine: 1750
starting turbine: 1751


In [ ]:
predictions_df.head(5)
then = dt.datetime.now

In [ ]:
average_mean_wind_directions_predictions = direction_predictions_df.mean(axis=1)
average_mean_wind_speed_predictions = speed_predictions_df.mean(axis=1)
print(average_mean_wind_directions_predictions)

In [ ]:
print(average_mean_wind_directions_real)

In [ ]:
combined_df = pd.concat([average_mean_wind_directions_predictions, average_mean_wind_directions_real, average_mean_wind_speeds_real, average_mean_wind_speed_predictions], axis=1)
print(combined_df)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(combined_df['WindDirectionMean'], color='blue', label='real')
plt.plot(combined_df[0], color='red', label='prediction')
plt.legend(loc='best')
plt.title('Real vs Predictions')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f = plt.figure(figsize=(20,5))
ax = f.add_subplot(221)
ax.plot(history, color='blue', label='Real')
ax.plot(predictions, color='red', label='Prediction')
f.autofmt_xdate()

ax.set_xlabel('Timestamp')
ax.set_ylabel('Wind Speed (m/s)')

ax.set_title('Real vs Predicted Average Wind Speed Values')
plt.legend()

plt.savefig('raw_single_wind_speed_prediction_plot.jpg')

In [ ]:
combined_df = combined_df.dropna()

plt.hist([combined_df['WindDirectionMean'], combined_df[0]], bins=list(range(0,360, 60)), label=['Real', 'Prediction'])
plt.legend(loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(combined_df['WindDirectionMean'], combined_df[0])
print('Test MSE: %.3f' % error)

In [ ]:
data["SpeedBin"] = data['WindSpeedMean'].apply(h.get_wind_speed_bin)
data["DirectionBin"] = data['WindDirectionMean'].apply(h.get_wind_direction_bin)

all_predictions["PredictedSpeedBin"] = all_predictions['WindSpeedMean'].apply(h.get_wind_speed_bin)
all_predictions["PredictedDirectionBin"] = all_predictions['WindDirectionMean'].apply(h.get_wind_direction_bin)

In [ ]:
comp = pd.DataFrame()

comp['WindSpeedMean'] = data['WindSpeedMean']
comp['WindDirectionMean'] = data['WindDirectionMean']

comp['PredictedWindSpeedMean'] = all_predictions['WindSpeedMean']
comp['PredictedWindDirectionMean'] = all_predictions['WindDirectionMean']

comp['SpeedBin'] = data['SpeedBin']
comp['DirectionBin'] = data['DirectionBin']
comp['PredictedSpeedBin'] = all_predictions["PredictedSpeedBin"] 
comp['PredictedDirectionBin'] = all_predictions["PredictedDirectionBin"] 

comp.dropna(inplace=True)
comp['SpeedBinEqual'] = comp['SpeedBin'] == comp["PredictedSpeedBin"]
comp['DirectionBinEqual'] = comp['DirectionBin'] == comp["PredictedDirectionBin"]

In [ ]:
wrong = comp.query('DirectionBinEqual == False or SpeedBinEqual == False')

error = len(wrong) / comp.shape[0] * 100

print("Percentage of bins wrong:")
print(error)

from sklearn.metrics import mean_squared_error
import math 

print("\nMSE for speed")
print(mean_squared_error(comp['WindSpeedMean'], comp['PredictedWindSpeedMean']))

print("\nMSE for direction")
print(math.degrees(mean_squared_error(comp['WindDirectionMean'].apply(math.radians), comp['PredictedWindDirectionMean'].apply(math.radians))))

In [ ]:
import numpy as np
comp['SpeedBinDifference'] = comp["PredictedSpeedBin"]- comp['SpeedBin']
comp['DirectionBinDifference'] = (comp['DirectionBin'] - comp["PredictedDirectionBin"]) / 30

In [ ]:
import seaborn as sns

g = sns.histplot(comp_2, x="SpeedBinDifference", y="DirectionBinDifference", discrete=True, cbar=True)
g.set_xticks(range(-12,12))#
g.set_yticks(range(-12,12))# <--- set the ticks first
g.set_title('Predicting Average Values Over All Turbines Bin Errors')

fig = g.get_figure()
fig.savefig("average_bin_errors.png")